In [2]:
import os
import pprint
import tempfile
import urllib

import absl
import tensorflow as tf
import tensorflow_model_analysis as tfma
tf.get_logger().propagate = False
pp = pprint.PrettyPrinter()

import tfx
from tfx.components import CsvExampleGen
from tfx.components import Evaluator
from tfx.components import ExampleValidator
from tfx.components import Pusher
from tfx.components import ResolverNode
from tfx.components import SchemaGen
from tfx.components import StatisticsGen
from tfx.components import Trainer
from tfx.components import Transform
from tfx.components.base import executor_spec
from tfx.components.trainer.executor import GenericExecutor
from tfx.dsl.experimental import latest_blessed_model_resolver
from tfx.orchestration import metadata
from tfx.orchestration import pipeline
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from tfx.proto import pusher_pb2
from tfx.proto import trainer_pb2
from tfx.types import Channel
from tfx.types.standard_artifacts import Model
from tfx.types.standard_artifacts import ModelBlessing
from tfx.utils.dsl_utils import external_input

%load_ext tfx.orchestration.experimental.interactive.notebook_extensions.skip

/home/kaimo/.local/lib/python3.7/site-packages/tfx/orchestration/data_types.py:191: UserWarning: RuntimeParameter is only supported on KubeflowDagRunner currently.
  warnings.warn('RuntimeParameter is only supported on KubeflowDagRunner '


In [130]:
%%skip_for_export
print('TensorFlow version: {}'.format(tf.__version__))
print('TFX version: {}'.format(tfx.__version__))

TensorFlow version: 2.1.0-dlenv_tfe
TFX version: 0.21.4
This cell will be skipped during export to pipeline.


In [4]:
## Set up pipeline paths

# This is the root directory for your TFX pip package installation.
_tfx_root = tfx.__path__[0]
# This is the directory containing the TFX Chicago Taxi Pipeline example.
_taxi_root = os.path.join(_tfx_root, 'examples/chicago_taxi_pipeline')
# This is the path where your model will be pushed for serving.
_serving_model_dir = os.path.join(tempfile.mkdtemp(), 'serving_model/taxi_simple')
# Set up logging
absl.logging.set_verbosity(absl.logging.INFO)

In [5]:
## Download example data
_data_root = tempfile.mkdtemp(prefix='tfx-data')
DATA_PATH = DATA_PATH = 'https://raw.githubusercontent.com/tensorflow/tfx/master/tfx/examples/chicago_taxi_pipeline/data/simple/data.csv'
_data_filepath = os.path.join(_data_root, 'data.csv')
urllib.request.urlretrieve(DATA_PATH, _data_filepath)

('/tmp/tfx-databrkkll78/data.csv', <http.client.HTTPMessage at 0x7f4fe78bce10>)

In [6]:
## Create the InteractiveContext
context = InteractiveContext()

## Run TFX components interactively

### ExampleGen

- Split data into training and evaluation sets
- Convert data into the tf.Example format
- Copy data into the _tfx_root directory for other components to access

In [7]:
example_gen = CsvExampleGen(input=external_input(_data_root))
context.run(example_gen)

INFO:absl:Running driver for CsvExampleGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for CsvExampleGen
INFO:absl:Generating examples.
INFO:absl:Using 1 process(es) for Beam pipeline execution.
INFO:absl:Processing input csv data /tmp/tfx-databrkkll78/* to TFExample.
INFO:absl:Examples generated.
INFO:absl:Running publisher for CsvExampleGen
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: Channel(
            type_name: Examples
            artifacts: [Artifact(type_name: Examples, uri: /tmp/tfx-interactive-2020-05-12T23_03_43.206029-iiig0ilg/CsvExampleGen/examples/1, id: 2)]
        ))

In [121]:
%%skip_for_export
artifact = example_gen.outputs['examples'].get()[0]
print(artifact.split_names, artifact.uri)

["train", "eval"] /tmp/tfx-interactive-2020-05-12T23_03_43.206029-iiig0ilg/CsvExampleGen/examples/1
This cell will be skipped during export to pipeline.


In [120]:
%%skip_for_export
# Get the URI of the output artifact representing the training examples, which is a directory
train_uri = os.path.join(example_gen.outputs['examples'].get()[0].uri, 'train')

# Get the list of files in this directory (all compressed TFRcord files)
tfrecords_filenames = [os.path.join(train_uri, name) for name in os.listdir(train_uri)]

# Create a `TFRecordDataset` to read these files
dataset = tf.data.TFRecordDataset(tfrecords_filenames, compression_type='GZIP')

# Iterate over the first 3 records and decode them
for tfrecord in dataset.take(3):
    serialized_example = tfrecord.numpy()
    example = tf.train.Example()
    example.ParseFromString(serialized_example)
    pp.pprint(example)

features {
  feature {
    key: "company"
    value {
      bytes_list {
        value: "Taxi Affiliation Services"
      }
    }
  }
  feature {
    key: "dropoff_census_tract"
    value {
    }
  }
  feature {
    key: "dropoff_community_area"
    value {
    }
  }
  feature {
    key: "dropoff_latitude"
    value {
    }
  }
  feature {
    key: "dropoff_longitude"
    value {
    }
  }
  feature {
    key: "fare"
    value {
      float_list {
        value: 27.049999237060547
      }
    }
  }
  feature {
    key: "payment_type"
    value {
      bytes_list {
        value: "Cash"
      }
    }
  }
  feature {
    key: "pickup_census_tract"
    value {
    }
  }
  feature {
    key: "pickup_community_area"
    value {
      int64_list {
        value: 60
      }
    }
  }
  feature {
    key: "pickup_latitude"
    value {
      float_list {
        value: 41.836151123046875
      }
    }
  }
  feature {
    key: "pickup_longitude"
    value {
      float_list {
        value: -87.

### StatisticGen

In [10]:
statistics_gen = StatisticsGen(examples=example_gen.outputs['examples'])
context.run(statistics_gen)

INFO:absl:Running driver for StatisticsGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for StatisticsGen
INFO:absl:Using 1 process(es) for Beam pipeline execution.
INFO:absl:Generating statistics for split train
INFO:absl:Statistics for split train written to /tmp/tfx-interactive-2020-05-12T23_03_43.206029-iiig0ilg/StatisticsGen/statistics/2/train.
INFO:absl:Generating statistics for split eval
INFO:absl:Statistics for split eval written to /tmp/tfx-interactive-2020-05-12T23_03_43.206029-iiig0ilg/StatisticsGen/statistics/2/eval.
/opt/conda/lib/python3.7/site-packages/tensorflow_data_validation/arrow/arrow_util.py:239: FutureWarning: Calling .data on ChunkedArray is provided for compatibility after Column was removed, simply drop this attribute
  types.FeaturePath([column_name]), column.data.chunk(0), weights):
INFO:absl:Running publisher for StatisticsGen
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: Channel(
            type_name: ExampleStatistics
            artifacts: [Artifact(type_name: ExampleStatistics, uri: /tmp/tfx-interactive-2020-05-12T23_03_43.206029-iiig0ilg/StatisticsGen/statistics/2, id: 3)]
        ))

In [122]:
%%skip_for_export
context.show(statistics_gen.outputs['statistics'])

This cell will be skipped during export to pipeline.


### SchemaGen

In [12]:
schema_gen = SchemaGen(statistics=statistics_gen.outputs['statistics'], \
                       infer_feature_shape=False)
context.run(schema_gen)

INFO:absl:Running driver for SchemaGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for SchemaGen
INFO:absl:Infering schema from statistics.


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


INFO:absl:Schema written to /tmp/tfx-interactive-2020-05-12T23_03_43.206029-iiig0ilg/SchemaGen/schema/3/schema.pbtxt.
INFO:absl:Running publisher for SchemaGen
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: Channel(
            type_name: Schema
            artifacts: [Artifact(type_name: Schema, uri: /tmp/tfx-interactive-2020-05-12T23_03_43.206029-iiig0ilg/SchemaGen/schema/3, id: 4)]
        ))

In [123]:
%%skip_for_export
context.show(schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'company',STRING,optional,single,'company'
'dropoff_census_tract',INT,optional,single,-
'dropoff_community_area',INT,optional,single,-
'dropoff_latitude',FLOAT,optional,single,-
'dropoff_longitude',FLOAT,optional,single,-
'fare',FLOAT,required,single,-
'payment_type',STRING,required,single,'payment_type'
'pickup_census_tract',INT,optional,single,-
'pickup_community_area',INT,required,single,-


,Values
Domain,
'company',"'0118 - 42111 Godfrey S.Awir', '0694 - 59280 Chinesco Trans Inc', '1085 - 72312 N and W Cab Co', '2092 - 61288 Sbeih company', '2192 - 73487 Zeymane Corp', '2192 - Zeymane Corp', '2733 - 74600 Benny Jona', '2809 - 95474 C & D Cab Co Inc.', '2823 - 73307 Seung Lee', '3011 - 66308 JBL Cab Inc.', '3152 - 97284 Crystal Abernathy', '3201 - C&D Cab Co Inc', '3201 - CID Cab Co Inc', '3253 - 91138 Gaither Cab Co.', '3385 - 23210 Eman Cab', '3385 - Eman Cab', '3623 - 72222 Arrington Enterprises', '3897 - 57856 Ilie Malec', '3897 - Ilie Malec', '4053 - Adwar H. Nikola', '4197 - 41842 Royal Star', '4615 - 83503 Tyrone Henderson', '4615 - Tyrone Henderson', '4623 - Jay Kim', '5006 - 39261 Salifu Bawa', '5074 - 54002 Ahzmi Inc', '5074 - Ahzmi Inc', '5129 - 87128', '5129 - 98755 Mengisti Taxi', '5129 - Mengisti Taxi', '585 - Valley Cab Co', '5864 - 73614 Thomas Owusu', '5864 - Thomas Owusu', '5874 - 73628 Sergey Cab Corp.', '5874 - Sergey Cab Corp.', '5997 - 65283 AW Services Inc.', '5997 - AW Services Inc.', '6057 - 24657 Richard Addo', '6488 - 83287 Zuha Taxi', '6574 - Babylon Express Inc.', '6742 - 83735 Tasha ride inc', 'Blue Ribbon Taxi Association Inc.', 'C & D Cab Co Inc', 'Chicago Elite Cab Corp.', 'Chicago Elite Cab Corp. (Chicago Carriag', 'Chicago Medallion Leasing INC', 'Chicago Medallion Management', 'Choice Taxi Association', 'Dispatch Taxi Affiliation', 'KOAM Taxi Association', 'Northwest Management LLC', 'Taxi Affiliation Services', 'Top Cab Affiliation'"
'payment_type',"'Cash', 'Credit Card', 'Dispute', 'No Charge', 'Pcard', 'Unknown'"


This cell will be skipped during export to pipeline.


### ExampleValidator

In [14]:
example_validator = ExampleValidator(statistics=statistics_gen.outputs['statistics'], \
                                     schema=schema_gen.outputs['schema'])
context.run(example_validator)

INFO:absl:Running driver for ExampleValidator
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for ExampleValidator
INFO:absl:Validating schema against the computed statistics.
INFO:absl:Validation complete. Anomalies written to /tmp/tfx-interactive-2020-05-12T23_03_43.206029-iiig0ilg/ExampleValidator/anomalies/4.
INFO:absl:Running publisher for ExampleValidator
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: ExampleValidator
    execution_id: 4
    outputs:
        anomalies: Channel(
            type_name: ExampleAnomalies
            artifacts: [Artifact(type_name: ExampleAnomalies, uri: /tmp/tfx-interactive-2020-05-12T23_03_43.206029-iiig0ilg/ExampleValidator/anomalies/4, id: 5)]
        ))

In [124]:
%%skip_for_export
context.show(example_validator.outputs['anomalies'])

,Anomaly short description,Anomaly long description
Feature name,,
'company',Unexpected string values,"Examples contain values missing from the schema: 3094 - 24059 G.L.B. Cab Co (<1%), 3319 - CD Cab Co (<1%), 4053 - 40193 Adwar H. Nikola (<1%), 4197 - Royal Star (<1%), 5006 - Salifu Bawa (<1%), 5724 - KYVI Cab Inc (<1%), 585 - 88805 Valley Cab Co (<1%), 6743 - Luhak Corp (<1%)."
'payment_type',Unexpected string values,Examples contain values missing from the schema: Prcard (<1%).
'pickup_census_tract',Column dropped,The feature was not present in any examples.


This cell will be skipped during export to pipeline.


### Transform

First, we define a few constants for feature engineering

In [16]:
_taxi_constants_module_file = 'taxi_constants.py'

In [17]:
%%skip_for_export
%%writefile {_taxi_constants_module_file}

# Categorical features are assumed to each have a maximum value in the dataset.
MAX_CATEGORICAL_FEATURE_VALUES = [24, 31, 12]

CATEGORICAL_FEATURE_KEYS = [
    'trip_start_hour', 'trip_start_day', 'trip_start_month',
    'pickup_census_tract', 'dropoff_census_tract', 'pickup_community_area',
    'dropoff_community_area'
]

DENSE_FLOAT_FEATURE_KEYS = ['trip_miles', 'fare', 'trip_seconds']

# Number of buckets used by tf.transform for encoding each feature.
FEATURE_BUCKET_COUNT = 10

BUCKET_FEATURE_KEYS = [
    'pickup_latitude', 'pickup_longitude', 'dropoff_latitude',
    'dropoff_longitude'
]

# Number of vocabulary terms used for encoding VOCAB_FEATURES by tf.transform
VOCAB_SIZE = 1000

# Count for out-of-vocab buckets in which unrecognized VOCAB_FEATURES are hashed.
OOV_SIZE = 10

VOCAB_FEATURE_KEY = [
    'payment_type',
    'company'
]

# keys
LABEL_KEY = 'tips'
FARE_KEY = 'fare'

def transformed_name(key):
    return key + '_xf'

Overwriting taxi_constants.py
This cell will be skipped during export to pipeline.


Next, we write a preprocessing_fn that takes in raw data as input, and returns transformed features that our model can train on

In [18]:
_taxi_transform_module_file = 'taxi_transform.py'

In [46]:
%%skip_for_export
%%writefile {_taxi_transform_module_file}

import tensorflow as tf
import tensorflow_transform as tft

import taxi_constants

_DENSE_FLOAT_FEATURE_KEYS = taxi_constants.DENSE_FLOAT_FEATURE_KEYS
_VOCAB_FEATURE_KEY = taxi_constants.VOCAB_FEATURE_KEY
_VOCAB_SIZE = taxi_constants.VOCAB_SIZE
_OOV_SIZE = taxi_constants.OOV_SIZE
_FEATURE_BUCKET_COUNT = taxi_constants.FEATURE_BUCKET_COUNT
_BUCKET_FEATURE_KEYS = taxi_constants.BUCKET_FEATURE_KEYS
_CATEGORICAL_FEATURE_KEYS = taxi_constants.CATEGORICAL_FEATURE_KEYS
_FARE_KEY = taxi_constants.FARE_KEY
_LABEL_KEY = taxi_constants.LABEL_KEY
_transformed_name = taxi_constants.transformed_name

def _fill_in_missing(x):
    """Replace missing values in a SparseTensor.
    Fills in missing values of `x` with `` or 0, and converts to a dense vector.
    Args:
        x: A `SparseTensor` of rank 2. Its dense shape should have size at most.  1
          in the second dimension.
    Returns:
        A rank 1 tensor where missing values of `x` have been filled in.
    """
    default_value = '' if x.dtype == tf.string else 0
    return tf.squeeze(
        tf.sparse.to_dense(
            tf.SparseTensor(x.indices, x.values, [x.dense_shape[0], 1]),
            default_value),
        axis=1)

def preprocessing_fn(inputs):
    """tf.transform's callback function for preprocessing inputs.
    Args:
        inputs: map from feature keys to raw not-yet-transformed features.
    Returns:
        Map from string feature key to transformed feature operations.
    """
    outputs = {}
    for key in _DENSE_FLOAT_FEATURE_KEYS:
        # Preserve this feature as a dense float, setting nan's to the mean.
        outputs[_transformed_name(key)] = tft.scale_to_z_score(_fill_in_missing(inputs[key]))
    for key in _VOCAB_FEATURE_KEY:
        # Build a vocabulary for this feature.
        outputs[_transformed_name(key)] = tft.compute_and_apply_vocabulary(
            _fill_in_missing(inputs[key]),
            top_k=_VOCAB_SIZE,
            num_oov_buckets=_OOV_SIZE)
    for key in _BUCKET_FEATURE_KEYS:
        outputs[_transformed_name(key)] = tft.bucketize(
            _fill_in_missing(inputs[key]),
            _FEATURE_BUCKET_COUNT,
            always_return_num_quantiles=False)
    for key in _CATEGORICAL_FEATURE_KEYS:
        outputs[_transformed_name(key)] = _fill_in_missing(inputs[key])
    
    # Was this passenger a big tipper?
    taxi_fare = _fill_in_missing(inputs[_FARE_KEY])
    tips = _fill_in_missing(inputs[_LABEL_KEY])
    outputs[_transformed_name(_LABEL_KEY)] = tf.where(
        tf.math.is_nan(taxi_fare),
        tf.cast(tf.zeros_like(taxi_fare), tf.int64),
        # Test if the tip was > 20% of fare.
        tf.cast(tf.greater(tips, tf.multiply(taxi_fare, tf.constant(0.2))), tf.int64))
    return outputs

Overwriting taxi_transform.py
This cell will be skipped during export to pipeline.


Now, we pass in this feature engineering code to the `Transform` component and run it to transform your data.

In [47]:
transform = Transform(
    examples=example_gen.outputs['examples'],
    schema=schema_gen.outputs['schema'],
    module_file=os.path.abspath(_taxi_transform_module_file)
)
context.run(transform)

INFO:absl:Running driver for Transform
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for Transform
INFO:absl:Using 1 process(es) for Beam pipeline execution.


INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
'Counter' object has no attribute 'name'
INFO:tensorflow:SavedModel written to: /tmp/tfx-interactive-2020-05-12T23_03_43.206029-iiig0ilg/Transform/transform_graph/13/.temp_path/tftransform_tmp/ceb703988aa94309976571b0552f240c/saved_model.pb
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
'Counter' object has no attribute 'name'
INFO:tensorflow:SavedModel written to: /tmp/tfx-interactive-2020-05-12T23_03_43.206029-iiig0ilg/Transform/transform_graph/13/.temp_path/tftransform_tmp/32c517e93dcb4146ac9eacbe9372f1b1/saved_model.pb
INFO:tensorflow:Saver not created because there are no variables in the graph to restore

INFO:absl:Running publisher for Transform
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: Transform
    execution_id: 13
    outputs:
        transform_graph: Channel(
            type_name: TransformGraph
            artifacts: [Artifact(type_name: TransformGraph, uri: /tmp/tfx-interactive-2020-05-12T23_03_43.206029-iiig0ilg/Transform/transform_graph/13, id: 22)]
        )
        transformed_examples: Channel(
            type_name: Examples
            artifacts: [Artifact(type_name: Examples, uri: /tmp/tfx-interactive-2020-05-12T23_03_43.206029-iiig0ilg/Transform/transformed_examples/13, id: 23)]
        ))

- `transform_graph` is the graph that can perform the preprocessing operations (this graph will be included in the serving and evaluation models).
- `transformed_examples` represents the preprocessed training and evaluation data.

### Trainer

In [49]:
_taxi_trainer_module_file = 'taxi_trainer.py'

In [68]:
%%skip_for_export
%%writefile {_taxi_trainer_module_file}

from typing import List, Text

import os
import absl
import datetime
import tensorflow as tf
import tensorflow_transform as tft

from tfx.components.trainer.executor import TrainerFnArgs

import taxi_constants

_DENSE_FLOAT_FEATURE_KEYS = taxi_constants.DENSE_FLOAT_FEATURE_KEYS
_VOCAB_FEATURE_KEY = taxi_constants.VOCAB_FEATURE_KEY
_VOCAB_SIZE = taxi_constants.VOCAB_SIZE
_OOV_SIZE = taxi_constants.OOV_SIZE
_FEATURE_BUCKET_COUNT = taxi_constants.FEATURE_BUCKET_COUNT
_BUCKET_FEATURE_KEYS = taxi_constants.BUCKET_FEATURE_KEYS
_CATEGORICAL_FEATURE_KEYS = taxi_constants.CATEGORICAL_FEATURE_KEYS
_FARE_KEY = taxi_constants.FARE_KEY
_LABEL_KEY = taxi_constants.LABEL_KEY
_transformed_name = taxi_constants.transformed_name
_MAX_CATEGORICAL_FEATURE_VALUES = taxi_constants.MAX_CATEGORICAL_FEATURE_VALUES

def _transformed_names(keys):
    return [_transformed_name(key) for key in keys]

def _gzip_reader_fn(filenames):
    """Small utility returning a record reader that can read gzip'ed file."""
    return tf.data.TFRecordDataset(filenames=filenames, compression_type='GZIP')

def _get_serve_tf_examples_fn(model, tf_transform_output):
    """Returns a function that parses a serialized tf.Example and applies TFT."""
    model.tft_layer = tf_transform_output.transform_features_layer()
    
    @tf.function
    def serve_tf_examples_fn(serialized_tf_examples):
        """Returns the output to be used in the serving signature."""
        feature_spec = tf_transform_output.raw_feature_spec()
        feature_spec.pop(_LABEL_KEY)
        parsed_features = tf.io.parse_example(serialized_tf_examples, feature_spec)
        
        transformed_features = model.tft_layer(parsed_features)
        transformed_features.pop(_transformed_name(_LABEL_KEY))
        
        return model(transformed_features)
    
    return serve_tf_examples_fn

def _input_fn(file_pattern: List[Text], \
              tf_transform_output: tft.TFTransformOutput, \
              batch_size: int = 200) -> tf.data.Dataset:
    """Generate features and label for tuning/training/
    
    Args:
        file_pattern: List of patths or patterns of input tfrecord files.
        tf_transform_output: A TFTransformOutput.
        batch_size: representing the number of consecutive elements of returned
          dataset to combine in a single batch
          
    Returns:
        A dataset that contains (features, indices) tuple where features is a 
          dictionary of Tensors, and indices is a single Tensor of label indices.
    """
    transformed_feature_spec = (tf_transform_output.transformed_feature_spec().copy())
    
    dataset = tf.data.experimental.make_batched_features_dataset(
        file_pattern=file_pattern,
        batch_size=batch_size,
        features=transformed_feature_spec,
        reader=_gzip_reader_fn,
        label_key=_transformed_name(_LABEL_KEY)
    )
    
    return dataset

def _build_keras_model(hidden_units: List[int] = None) -> tf.keras.Model:
    """Creates a DNN Keras model for classifying taxi data.
    
    Args:
        hidden_units: [int], the layer sizes of the DNN (input layer first).
        
    Returns:
        A Keras Model.
    """
    real_valued_columns = [
        tf.feature_column.numeric_column(key, shape=())
        for key in _transformed_names(_DENSE_FLOAT_FEATURE_KEYS)
    ]
    categorical_columns = [
        tf.feature_column.categorical_column_with_identity(
            key, num_buckets=_VOCAB_SIZE+_OOV_SIZE, default_value=0)
        for key in _transformed_names(_VOCAB_FEATURE_KEY)
    ]
    categorical_columns += [
        tf.feature_column.categorical_column_with_identity(
            key, num_buckets=_FEATURE_BUCKET_COUNT, default_value=0)
        for key in _transformed_names(_BUCKET_FEATURE_KEYS)
    ]
    categorical_columns += [
        tf.feature_column.categorical_column_with_identity(
            key, num_buckets=num_buckets, default_value=0)
        for key, num_buckets in zip(_transformed_names(_CATEGORICAL_FEATURE_KEYS),
                                    _MAX_CATEGORICAL_FEATURE_VALUES)
    ]
    indicator_column = [
        tf.feature_column.indicator_column(categorical_column)
        for categorical_column in categorical_columns
    ]
    
    model = _wide_and_deep_classifier(
        wide_columns=indicator_column,
        deep_columns=real_valued_columns,
        dnn_hidden_units=hidden_units or [100, 70, 50, 25]
    )
    return model

def _wide_and_deep_classifier(wide_columns, deep_columns, dnn_hidden_units) -> tf.keras.Model:
    """Build a simple keras and deep model.
    
    Args:
        wide_columns: Feature columns wrapped in indicator_column for wide (linear)
          part of the model.
        deep_columns:  Feature columns for deep part of the model.
        dnn_hidden_units: [int], the layer sizes of the hidden DNN.
        
    Returns:
        A wide and Deep Keras model
    """
    input_layers = {
        colname: tf.keras.layers.Input(name=colname, shape=(), dtype=tf.float32)
        for colname in _transformed_names(_DENSE_FLOAT_FEATURE_KEYS)
    }
    input_layers.update({
        colname: tf.keras.layers.Input(name=colname, shape=(), dtype='int32')
        for colname in _transformed_names(_VOCAB_FEATURE_KEY + 
                                          _BUCKET_FEATURE_KEYS + 
                                          _CATEGORICAL_FEATURE_KEYS)
    })
    
    deep = tf.keras.layers.DenseFeatures(deep_columns)(input_layers)
    for numnodes in dnn_hidden_units:
        deep = tf.keras.layers.Dense(numnodes)(deep)
    wide = tf.keras.layers.DenseFeatures(wide_columns)(input_layers)
    
    output = tf.keras.layers.Dense(1, activation='sigmoid')(tf.keras.layers.concatenate([deep, wide]))
    
    model = tf.keras.Model(input_layers, output)
    model.compile(
        loss='binary_crossentropy',
        optimizer=tf.keras.optimizers.Adam(lr=0.001),
        metrics=[tf.keras.metrics.BinaryAccuracy()]
    )
    model.summary(print_fn=absl.logging.info)
    return model

# TFX Trainer will call this function.
def run_fn(fn_args: TrainerFnArgs):
    """Train the model based on given args.
    
    Args:
        fn_args: Holds args used to train the model as name/value pairs.
    """
    # Number of nodes in the first layer of the DNN
    first_dnn_layer_size = 100
    num_dnn_layers = 4
    dnn_decay_factor = 0.7
    
    tf_transform_output = tft.TFTransformOutput(fn_args.transform_output)
    
    train_dataset = _input_fn(fn_args.train_files, tf_transform_output, 40)
    eval_dataset = _input_fn(fn_args.eval_files, tf_transform_output, 40)
    
    model = _build_keras_model(
        # Construct layers sizes with exponential decay
        hidden_units=[
            max(2, int(first_dnn_layer_size * dnn_decay_factor**i))
            for i in range(num_dnn_layers)
        ]
    )
    
    # This log path might change in the future
    log_dir = os.path.join(os.path.dirname(fn_args.serving_model_dir), 'logs')
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, update_freq='batch')
    model.fit(
        train_dataset,
        steps_per_epoch=fn_args.train_steps,
        validation_data=eval_dataset,
        validation_steps=fn_args.eval_steps,
        callbacks=[tensorboard_callback]
    )
    
    signatures = {
        'serving_default':
            _get_serve_tf_examples_fn(model, tf_transform_output)\
                .get_concrete_function(tf.TensorSpec(shape=[None], dtype=tf.string, name='examples')),
    }
    model.save(fn_args.serving_model_dir, save_format='tf', signatures=signatures)

Overwriting taxi_trainer.py
This cell will be skipped during export to pipeline.


Now we pass in this model code to the `Trainer` component and run it to train the model.

In [74]:
trainer = Trainer(
    examples=transform.outputs['transformed_examples'],
    transform_graph=transform.outputs['transform_graph'],
    schema=schema_gen.outputs['schema'],
    module_file=os.path.abspath(_taxi_trainer_module_file),
    train_args=trainer_pb2.TrainArgs(num_steps=10000),
    eval_args=trainer_pb2.EvalArgs(num_steps=5000),
    custom_executor_spec=executor_spec.ExecutorClassSpec(GenericExecutor)
)
context.run(trainer)

INFO:absl:Running driver for Trainer
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for Trainer
INFO:absl:Training model.
INFO:absl:Model: "model_3"
INFO:absl:__________________________________________________________________________________________________
INFO:absl:Layer (type)                    Output Shape         Param #     Connected to                     
INFO:absl:==================================================================================================
INFO:absl:company_xf (InputLayer)         [(None,)]            0                                            
INFO:absl:__________________________________________________________________________________________________
INFO:absl:dropoff_census_tract_xf (InputL [(None,)]            0                                            
INFO:absl:__________________________________________________________________________________________________
INFO:absl:dropoff_community_area_xf (Inpu [(None,)]  

INFO:absl:                                                                 trip_start_hour_xf[0][0]         
INFO:absl:                                                                 trip_start_month_xf[0][0]        
INFO:absl:__________________________________________________________________________________________________
INFO:absl:concatenate_3 (Concatenate)     (None, 2161)         0           dense_18[0][0]                   
INFO:absl:                                                                 dense_features_7[0][0]           
INFO:absl:__________________________________________________________________________________________________
INFO:absl:dense_19 (Dense)                (None, 1)            2162        concatenate_3[0][0]              
INFO:absl:==================================================================================================
INFO:absl:Total params: 14,706
INFO:absl:Trainable params: 14,706
INFO:absl:Non-trainable params: 0
INFO:absl:__________________

Train for 10000 steps, validate for 5000 steps
10000/10000 [==============================] - 117s 12ms/step - loss: 0.2457 - binary_accuracy: 0.8766 - val_loss: 0.2091 - val_binary_accuracy: 0.8954
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: LIVE_VARS_IN
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: LIVE_VARS_IN
value: "\n\013\n\tConst_7:0\022-vocab_compute_and_apply_vocabulary_vocabulary"

value: "\n\013\n\tConst_9:0\022/vocab_compute_and_apply_vocabulary_1_vocabulary"

INFO:tensorflow:Saver not created because there are no variables in the graph to restore
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: /tmp/tfx-interactive-2020-05-12T23_03_43.206029-iiig0ilg/Trainer/model/19/

INFO:absl:Training complete. Model written to /tmp/tfx-interactive-2020-05-12T23_03_43.206029-iiig0ilg/Trainer/model/19/serving_model_dir
INFO:absl:Running publisher for Trainer
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: Trainer
    execution_id: 19
    outputs:
        model: Channel(
            type_name: Model
            artifacts: [Artifact(type_name: Model, uri: /tmp/tfx-interactive-2020-05-12T23_03_43.206029-iiig0ilg/Trainer/model/19, id: 29)]
        ))

Analyze Training with TensorBoard

In [125]:
%%skip_for_export
model_artifact_dir = trainer.outputs['model'].get()[0].uri
pp.pprint(os.listdir(model_artifact_dir))
model_dir = os.path.join(model_artifact_dir, 'serving_model_dir')
pp.pprint(os.listdir(model_dir))

['serving_model_dir', 'logs']
['variables', 'assets', 'saved_model.pb']
This cell will be skipped during export to pipeline.


In [80]:
%%skip_for_export

log_dir = os.path.join(model_artifact_dir, 'logs')

%load_ext tensorboard
%tensorboard --logdir {log_dir}

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 3163), started 0:00:45 ago. (Use '!kill 3163' to kill it.)

This cell will be skipped during export to pipeline.


### Evaluator

In [86]:
eval_config = tfma.EvalConfig(
    model_specs=[
        # This assumes a serving model with signature 'serving_default'. If
        # using estimator based EvalSavedModel, add signature_name: 'eval' and 
        # remove the label_key
        tfma.ModelSpec(label_key='tips')],
    metrics_specs=[
        tfma.MetricsSpec(
            # The metrics added here are in addition to those saved with the
            # model (assuming either a keras model or EvalSavedModel is used).
            # Any metrics added into the saved model (for example using
            # model.compile(..., metrics=[...]), etc) will be computed
            # automatically.
            metrics=[tfma.MetricConfig(class_name='ExampleCount')],
            # To add validation thresholds for metrics saved with the model,
            # add them keyed by metric name to the thresholds map.
            thresholds={
                'binary_accuracy': tfma.MetricThreshold(
                    value_threshold=tfma.GenericValueThreshold(
                        lower_bound={'value': 0.5}),
                    change_threshold=tfma.GenericChangeThreshold(
                        direction=tfma.MetricDirection.HIGHER_IS_BETTER,
                        absolute={'value': -1e-10})
                )
            }
        )
    ],
    slicing_specs=[
        # An empty slice spec means the overall slice, i.e. the whole dataset.
        tfma.SlicingSpec(),
        # Data can be sliced along a feature column. In this case, data is
        # sliced along feature column trip_start_hour
        tfma.SlicingSpec(feature_keys=['trip_start_hour'])
    ]
)

In [88]:
# Use TFMA to compute a evaluation statistics over features of a model and
# validate them against a baseline.

# The model resolver is only required if performing model validation in addition
# to evaluation. In this case we validate against the latest blessed model. If
# no model has been blessed before (as in this case) the evaluator will make our
# candidate the first blessed model.
model_resolver = ResolverNode(
    instance_name='latest_blessed_model_resolver',
    resolver_class=latest_blessed_model_resolver.LatestBlessedModelResolver,
    model=Channel(type=Model),
    model_blessing=Channel(type=ModelBlessing)
)
context.run(model_resolver)

evaluator = Evaluator(
    examples=example_gen.outputs['examples'],
    model=trainer.outputs['model'],
    baseline_model=model_resolver.outputs['model'],
    # Change threshold will be ignored if there is no baseline (first run)
    eval_config=eval_config
)
context.run(evaluator)

INFO:absl:Running driver for ResolverNode.latest_blessed_model_resolver
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running publisher for ResolverNode.latest_blessed_model_resolver
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running driver for Evaluator
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for Evaluator
INFO:absl:Request was made to ignore the baseline ModelSpec and any change thresholds. This is likely because a baseline model was not provided: updated_config=
model_specs {
  label_key: "tips"
}
slicing_specs {
}
slicing_specs {
  feature_keys: "trip_start_hour"
}
metrics_specs {
  metrics {
    class_name: "ExampleCount"
    threshold {
    }
  }
  thresholds {
    key: "binary_accuracy"
    value {
      value_threshold {
        lower_bound {
          value: 0.5
        }
      }
    }
  }
}

INFO:absl:Using /tmp/tfx-interactive-2020-05-12T23_03_43.206029-iiig0ilg/Trainer/model/19/serving_model_

ExecutionResult(
    component_id: Evaluator
    execution_id: 21
    outputs:
        evaluation: Channel(
            type_name: ModelEvaluation
            artifacts: [Artifact(type_name: ModelEvaluation, uri: /tmp/tfx-interactive-2020-05-12T23_03_43.206029-iiig0ilg/Evaluator/evaluation/21, id: 30)]
        )
        blessing: Channel(
            type_name: ModelBlessing
            artifacts: [Artifact(type_name: ModelBlessing, uri: /tmp/tfx-interactive-2020-05-12T23_03_43.206029-iiig0ilg/Evaluator/blessing/21, id: 31)]
        ))

In [89]:
%%skip_for_export
context.show(evaluator.outputs['evaluation'])

This cell will be skipped during export to pipeline.


In [96]:
%%skip_for_export
!ls -l {evaluator.outputs['blessing'].get()[0].uri}

total 0
-rw-r--r-- 1 kaimo kaimo 0 May 13 01:54 BLESSED
This cell will be skipped during export to pipeline.


In [97]:
%%skip_for_export
PATH_TO_RESULT = evaluator.outputs['evaluation'].get()[0].uri
tfma.load_validation_result(PATH_TO_RESULT)

validation_ok: true

This cell will be skipped during export to pipeline.


### Pusher

The `Pusher` component is usually at the end of a TFX pipeline. It checks whether a model has passed validation, and if so, exports the model to `_serving_model_dir`.

In [99]:
pusher = Pusher(
    model=trainer.outputs['model'],
    model_blessing=evaluator.outputs['blessing'],
    push_destination=pusher_pb2.PushDestination(
        filesystem=pusher_pb2.PushDestination.Filesystem(
            base_directory=_serving_model_dir))
)
context.run(pusher)

INFO:absl:Running driver for Pusher
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for Pusher
INFO:absl:Model pushing.
INFO:absl:Model version is 1589335276
INFO:absl:Model written to /tmp/tfx-interactive-2020-05-12T23_03_43.206029-iiig0ilg/Pusher/pushed_model/22.
INFO:absl:Model written to serving path /tmp/tmp0_nhe1xk/serving_model/taxi_simple/1589335276.
INFO:absl:Model pushed to /tmp/tmp0_nhe1xk/serving_model/taxi_simple/1589335276.
INFO:absl:Running publisher for Pusher
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: Pusher
    execution_id: 22
    outputs:
        pushed_model: Channel(
            type_name: PushedModel
            artifacts: [Artifact(type_name: PushedModel, uri: /tmp/tfx-interactive-2020-05-12T23_03_43.206029-iiig0ilg/Pusher/pushed_model/22, id: 32)]
        ))

In [100]:
%%skip_for_export
pusher.outputs

{'pushed_model': Channel(
    type_name: PushedModel
    artifacts: [Artifact(type_name: PushedModel, uri: /tmp/tfx-interactive-2020-05-12T23_03_43.206029-iiig0ilg/Pusher/pushed_model/22, id: 32)]
)}

This cell will be skipped during export to pipeline.


In particular, the Pusher will export your model in the SavedModel format, which looks like this:

In [106]:
%%skip_for_export

push_uri = pusher.outputs['pushed_model'].get()[0].uri
latest_version = max(os.listdir(push_uri))
latest_version_path = os.path.join(push_uri, latest_version)
model = tf.saved_model.load(latest_version_path)

for item in model.signatures.items():
    pp.pprint(item)

('serving_default',
 <tensorflow.python.saved_model.load._WrapperFunction object at 0x7f4b8b725a50>)
This cell will be skipped during export to pipeline.


## Export to pipeline

To export the contents of this notebook as a pipeline to be orchestrated with Airflow or Beam, follow the instructions below

### Select an orchestrator

In [111]:
_runner_type = 'beam'
_pipeline_name = 'chicage_taxi_{:s}'.format(_runner_type)

### Set up paths for the pipeline

In [126]:
_notebook_filepath = os.path.join(os.getcwd(), 'TFX-Keras-Component-Tutorial.ipynb')

_tfx_root = os.path.join(os.environ['HOME'], 'tfx')
_taxi_root = os.path.join(os.environ['HOME'], 'taxi')

_serving_model_dir = os.path.join(_taxi_root, 'serving_model')
_data_root = os.path.join(_taxi_root, 'data', 'simple')

_pipeline_root = os.path.join(_tfx_root, 'pipeline', _pipeline_name)
_metadata_path = os.path.join(_tfx_root, 'metadata', _pipeline_name, 'metadata.db')

### Choose components to include in the pipeline

In [119]:
components = [
    example_gen, statistics_gen, schema_gen, example_validator, transform, trainer, evaluator, pusher
]

### Generate pipeline files

In [131]:
%%skip_for_export
if get_ipython().magics_manager.auto_magic:
    print('Warning: %automagic is ON. Line magics specified without the % prefix '
          'will not be scrubbed during export to pipeline.')
    
_pipeline_export_filepath = 'export_{:s}.py'.format(_pipeline_name)
context.export_to_pipeline(
    notebook_filepath=_notebook_filepath,
    export_filepath=_pipeline_export_filepath,
    runner_type=_runner_type
)

INFO:absl:Exporting contents of /home/kaimo/Hands-on-TensorFlow-Extend/TFX-Keras-Component-Tutorial.ipynb to export_chicage_taxi_beam.py with beam runner.
INFO:absl:17 cell(s) marked with "%%skip_for_export", skipped.


This cell will be skipped during export to pipeline.
